In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import sys

sys.path.append("../../notebook_styling")
import psrc_theme

input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
valid_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))

# create connection
conn = create_engine('sqlite:///../../../../inputs/db/'+input_config['db_name'])

# summary data location
csv_path = os.path.join(valid_config['model_dir'], 'outputs/validation')

# table format
pd.options.display.float_format = '{:0,.2f}'.format
format_percent = "{:,.2%}".format

# Link Expansion Analysis
Testing 2023 base year with Lynnwood extension and 2-Line connection to Seattle

### Line Level Totals


In [2]:
df_obs = pd.read_sql(
    "SELECT * FROM light_rail_station_boardings WHERE year IN (2023, 2025)",
    con=conn,
)
df_obs.rename(columns={"boardings": "observed_boardings"}, inplace=True)

df_obs
df_obs_pivot = pd.pivot_table(df_obs, values='observed_boardings', columns='year', index='station_name', aggfunc=np.sum)
df_obs_pivot = df_obs_pivot.fillna(0)
df_obs_pivot = df_obs_pivot.reset_index()
_df = df_obs.loc[df_obs['year']==2025,['station_name','emme_node']]
df_obs_pivot = df_obs_pivot.merge(_df, on=['station_name'])

df = pd.read_csv(os.path.join(valid_config['model_dir'], 'outputs/transit/boardings_by_stop.csv'))
df = df[df["i_node"].isin(df_obs["emme_node"])]
df = df.merge(df_obs_pivot, left_on="i_node", right_on="emme_node")
df.rename(columns={"total_boardings": "Model Boardings"}, inplace=True)
df.index = df["station_name"]
df_total = df.copy()[["Model Boardings",2023,2025, ]]
df_total.loc["Total", ["Model Boardings", 2023,2025]] = (
    df[["Model Boardings", 2023,2025]].sum().values
)
df_total
df_total.rename(columns={2023: 'Observed Spring 2023 (Pre-Lynnwood)', 2025: 'Observed Fall 2024 (Post-Lynnwood)' }, 
               inplace=True)
line_1_stations = ["Lynnwood Station","Mountlake Terrace Station","185th St Station","145th St Station",
                "Northgate Station",'Roosevelt Station','University District Station','Husky Stadium',
              'Capitol Hill','Westlake','Symphony',
              "Pioneer Square","Int'l District",
                 'Stadium','SODO','Beacon Hill','Mount Baker','Columbia City','Othello','Rainier Beach',
                "Tukwila Int'l Blvd",'SeaTac/Airport','Angle Lake']
station_list = [i for i in line_1_stations if i in df_total.index]
df_1_line = df_total.loc[station_list]


line_2_stations = ['South Bellevue Station','East Main Station','Bellevue Station','Wilburton Station',
             'Spring District/120th Station','Bel-Red/130th Station','Overlake Village Station',
             'Redmond Technology Station']
station_list = [i for i in line_2_stations if i in df_total.index]
if len(station_list) > 0:
    df_2_line = df_total.loc[line_2_stations]
else:
    df_2_line = None

C:\Users\Modeller\AppData\Local\Temp\ipykernel_31524\1347007469.py:8: FutureWarning:

The provided callable <function sum at 0x0000019FBE337920> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



In [3]:
df_total.loc[df_total.index.isin(line_1_stations),'Line'] = '1-Line'
df_total.loc[df_total.index.isin(line_2_stations),'Line'] = '2-Line'
df_total.groupby('Line').sum()

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
Line,,,
1-Line,"251,548.93","79,522.00","92,766.00"
2-Line,"57,214.58",0.00,"5,422.00"


### 1-Line Lynnwood to Angle Lake

In [4]:
df_1_line

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,
Lynnwood Station,"10,465.35",0.00,"5,146.00"
Mountlake Terrace Station,"9,631.51",0.00,"1,393.00"
185th St Station,"8,284.21",0.00,957.00
145th St Station,"6,871.06",0.00,"1,020.00"
Northgate Station,"9,372.04","10,384.00","4,150.00"
Roosevelt Station,"11,237.43","4,349.00","3,693.00"
University District Station,"20,124.84","7,563.00","6,004.00"
Husky Stadium,"11,761.86","6,134.00","5,122.00"
Capitol Hill,"15,526.31","7,667.00","8,207.00"


In [5]:
fig = px.line(df_1_line)
fig.update_layout(height=500, width=900, font=dict(size=11),
                  yaxis_tickformat = '.00f')
fig.show()

### 2-Line

In [6]:
df_2_line

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,
South Bellevue Station,"4,433.69",0.00,"1,031.00"
East Main Station,"14,179.63",0.00,234.00
Bellevue Station,"18,085.06",0.00,"1,212.00"
Wilburton Station,"3,969.58",0.00,293.00
Spring District/120th Station,"1,460.26",0.00,424.00
Bel-Red/130th Station,"5,639.00",0.00,311.00
Overlake Village Station,"6,525.04",0.00,418.00
Redmond Technology Station,"2,922.33",0.00,"1,499.00"


In [7]:
if df_2_line is not None:
    fig = px.line(df_2_line)
    fig.update_layout(height=500, width=900, font=dict(size=11),
                    yaxis_tickformat = '.00f')
    fig.show()